In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
path = os.path.join(os.getcwd(),'files')
datasets = os.path.join(path,'datasets')

In [3]:
os.listdir(datasets)

['test_data.parquet',
 'test_for_participants.csv',
 'train_data.parquet',
 'train_pairs_w_target.parquet']

In [4]:
pairs = pd.read_parquet(os.path.join(datasets,'train_pairs_w_target.parquet'))

In [5]:
pairs

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289
2,1.0,53602615,587809782
3,1.0,53888651,89598677
4,0.0,56930698,551526166
...,...,...,...
306535,0.0,817327230,822083612
306536,0.0,817560551,818069912
306537,0.0,817854719,817857267
306538,0.0,820036017,820037019


In [25]:
def increase_db(db,niters=5):
    db.target = db.target.apply(int)
    indexes = db.variantid1.value_counts().index
    variant_ids1 = indexes[:len(indexes)//2]
    
    for idx in tqdm(variant_ids1):
        pairs = db[db.variantid1==idx].merge(db,left_on='variantid2', right_on='variantid1')
        pairs['target'] = pairs['target_x']&pairs['target_y']
        pairs['variantid1'] = pairs['variantid1_x']
        pairs['variantid2'] = pairs['variantid2_y']
        pairs = pairs[['target', 'variantid1','variantid2']]
        db =  pd.concat([db,pairs])
    db = db.reset_index()
    db.drop('index',axis=1,inplace=True)
        
    return db

In [26]:
db = increase_db(pairs)

100%|████████████████████████████████████████████████████████████████████████| 123069/123069 [3:28:59<00:00,  9.81it/s]


In [43]:
db = db.drop_duplicates()

In [44]:
db.to_parquet('train_pairs_ext.parquet')

In [47]:
db.target.value_counts()

1    243842
0    224917
Name: target, dtype: int64

In [48]:
pairs.target.value_counts()

0    171527
1    135013
Name: target, dtype: int64